In [1]:
import numpy as np
import pandas as pd
import os

In [3]:
#import libraries 
import tensorflow as tf
import keras
#from kerastuner.tuners import RandomSearch
#from kerastuner.engine.hyperparameters import HyperParameter as hp
from keras.layers import Dense,Dropout,Activation,Add,MaxPooling2D,Conv2D,Flatten,BatchNormalization,MaxPool2D
from keras.models import Sequential
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.applications import VGG19

In [4]:
from PIL import Image
image = Image.open('../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/train/ok_front/cast_ok_0_1.jpeg')
numpydata = np.asarray(image)
numpydata.shape

In [5]:
#import train data
train_datagen = ImageDataGenerator(rescale=1/255,
                                   shear_range = 0.3,
                                   zoom_range = 0.3,horizontal_flip = True,
                                   vertical_flip =  True , 
                                   rotation_range=60,validation_split=0.2)


train_data = train_datagen.flow_from_directory('../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/train',
                                                 target_size = (244, 244),
                                                 class_mode="categorical",subset="training",
                                               batch_size=8,
                                                 shuffle=False)
#import test data

test_datagen = ImageDataGenerator(rescale = 1/255)
test_data = test_datagen.flow_from_directory("../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/test",
                                                           batch_size=8,
                                                           target_size=(244,244),
                                                           class_mode="categorical",
                                                           shuffle=True)
val_data = train_datagen.flow_from_directory('../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/train',
                                                           batch_size=8,
                                                           target_size=(244,244),
                                                           class_mode="categorical",
                                                           subset="validation",
                                                           shuffle=True)


In [6]:
#build our model 
vgg_model =  VGG19(include_top=False , 
                   weights='imagenet',
                   input_shape=(244, 244,3),
                   classifier_activation = 'softmax',
                   pooling="max",
                   classes=2)
vgg_model.trainable= False
vgg_model.summary()

In [7]:
model = Sequential([
                                  vgg_model,
                                  layers.Dense(1000, activation = 'relu'),
                                  layers.Dense(500, activation = 'relu'),
                                  layers.Dropout(0.25),
                                  layers.Dense(100, activation = 'relu'),
                                  layers.Dense(2, activation = 'softmax') 
])

In [8]:
model.summary()

In [9]:
#compile my model
model.compile(optimizer="adam",loss="categorical_crossentropy", metrics=["accuracy"])

In [11]:
#fit our model
history = model.fit(train_data,validation_data= val_data,epochs = 10,verbose=2)

In [13]:
#evulate model 
model.evaluate(test_data)

In [15]:
#predict test data
y_prediction =  model.predict(test_data)
y_prediction =  np.argmax(y_prediction,axis = 1 )

print(y_prediction)

In [29]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

#accuracy plot
plt.plot(epochs, acc, label='Training Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.figure()
#loss plot
plt.plot(epochs, loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [39]:
classes_labels = {0:"def_front",1:"ok_front"}

In [41]:
# test Example 1    Actual - Defected 
import cv2
image_path = "../input/real-life-industrial-dataset-of-casting-product/casting_512x512/casting_512x512/def_front/cast_def_0_1055.jpeg"
img= cv2.imread(image_path)
img=cv2.resize(img, (244, 244))
img = np.expand_dims(img, axis=0)
prediction = model.predict(img)
prediction = np.argmax(prediction,axis=1)
print("Predicted-",prediction)
print(classes_labels[prediction[0]])

plt.imshow(new_img)

In [43]:
#test_example 2  Actual - Not Defected 
image_path = "../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/train/ok_front/cast_ok_0_1016.jpeg"
img= cv2.imread(image_path)
img=cv2.resize(img, (244, 244))
img = np.expand_dims(img, axis=0)
prediction = model.predict(img)
prediction = np.argmax(prediction,axis=1)
print("Predicted-",prediction)
print(classes_labels[prediction[0]])

plt.imshow(new_img)

In [45]:
#test_example 3    Actual - Not Defected 
image_path = "../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/train/ok_front/cast_ok_0_1019.jpeg"
img= cv2.imread(image_path)
img=cv2.resize(img, (244, 244))
img = np.expand_dims(img, axis=0)
prediction = model.predict(img)
prediction = np.argmax(prediction,axis=1)
print("Predicted-",prediction)
print(classes_labels[prediction[0]])

plt.imshow(new_img)

In [46]:
#test_example 4   Actual - Defected 
image_path = "../input/real-life-industrial-dataset-of-casting-product/casting_512x512/casting_512x512/def_front/cast_def_0_1188.jpeg"
img= cv2.imread(image_path)
img=cv2.resize(img, (244, 244))
img = np.expand_dims(img, axis=0)
prediction = model.predict(img)
prediction = np.argmax(prediction,axis=1)
print("Predicted-",prediction)
print(classes_labels[prediction[0]])

plt.imshow(new_img)

In [ ]:
# to save model
model.save("task_model_casting.h5") 